In [1]:
import torch
import torchvision.models as models
import torchvision.transforms as transform
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, WeightedRandomSampler
import torch.optim as optim
import torch.nn as nn
import numpy as np

In [ ]:
IMAGE_SIZE = 224
BATCH_SIZE = 16
NUM_WORKERS = 2
MEAN = [0.485, 0.456, 0.406]
STD  = [0.229, 0.224, 0.225]

train_transforms = transform.Compose([
    transform.RandomResizedCrop(IMAGE_SIZE, scale=(0.8, 1.0)), 
    transform.RandomHorizontalFlip(),                    
    transform.RandomVerticalFlip(),
    transform.RandomRotation(15),                         
    transform.ColorJitter(brightness=0.2, contrast=0.2),
    transform.ToTensor(),                               
    transform.Normalize(MEAN, STD)                    
])

val_transforms = transform.Compose([
    transform.Resize(256),
    transform.CenterCrop(IMAGE_SIZE),        
    transform.ToTensor(),
    transform.Normalize(MEAN, STD)
])

In [ ]:
train_dataset = ImageFolder(root='../../dataset-dapa//Train/', transform=train_transforms)
val_dataset = ImageFolder(root='../../dataset-dapa/val/', transform=train_transforms)

class_counts = np.bincount([label for _, label in train_dataset.samples])
class_weights = 1. / torch.tensor(class_counts, dtype=torch.float)

sample_weights = [class_weights[label] for _, label in train_dataset.samples]
sampler = WeightedRandomSampler(sample_weights, num_samples=len(sample_weights), replacement=True)

train_loader = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=NUM_WORKERS,
    pin_memory=True
)


val_loader = DataLoader(
    val_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=NUM_WORKERS,
    pin_memory=True
)

print(f"Classes detected: {train_dataset.classes}")

Classes detected: ['algal_spot', 'brown_blight', 'gray_blight', 'healthy', 'helopeltis', 'red-rust', 'red-spider-infested', 'red_spot', 'white-spot']


In [ ]:
model = models.densenet201(weights=models.DenseNet201_Weights.IMAGENET1K_V1)
features = model.features
features.requires_grad_(False) 

Sequential(
  (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu0): ReLU(inplace=True)
  (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (denseblock1): _DenseBlock(
    (denselayer1): _DenseLayer(
      (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu1): ReLU(inplace=True)
      (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu2): ReLU(inplace=True)
      (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    )
    (denselayer2): _DenseLayer(
      (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu1): ReLU(inplace=True)
      (conv1): Conv2d(96, 128, ke

In [ ]:
num_classes = len(train_dataset.classes)
model.classifier = nn.Sequential(
    nn.Dropout(0.3),
    nn.Linear(model.classifier.in_features, num_classes)
)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print('is cuda available?', torch.cuda.is_available())
model = model.to(device)


is cuda available? True


In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.classifier.parameters(), lr=1e-3) 

In [7]:
def train_one_epoch(model, loader, optimizer, criterion, device):
    model.train()
    running_loss, correct, total = 0.0, 0, 0
    for images, labels in loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        preds = outputs.argmax(dim=1)
        correct += (preds == labels).sum().item()
        total += images.size(0)
    return running_loss/total, correct/total

def validate(model, loader, criterion, device):
    model.eval()
    val_loss, correct, total = 0.0, 0, 0
    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)

            val_loss += loss.item() * images.size(0)
            preds = outputs.argmax(dim=1)
            correct += (preds == labels).sum().item()
            total += images.size(0)
    return val_loss/total, correct/total


In [ ]:
num_epochs = 30
best_val_loss = float('inf')

print(f"Total batches: {len(train_loader)}")
for epoch in range(num_epochs):
    train_loss, train_acc = train_one_epoch(model, train_loader, optimizer, criterion, device)
    val_loss, val_acc     = validate(model, val_loader, criterion, device)

    print(f"Epoch {epoch+1}/{num_epochs}: "
          f"Train loss {train_loss:.4f}, acc {train_acc:.4f} | "
          f"Val   loss {val_loss:.4f}, acc {val_acc:.4f}")

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), "best_densenet201.pth")


Total batches: 290
Epoch 1/30: Train loss 0.9940, acc 0.6956 | Val   loss 0.4941, acc 0.8717
Epoch 2/30: Train loss 0.5772, acc 0.8145 | Val   loss 0.4204, acc 0.8657
Epoch 3/30: Train loss 0.5042, acc 0.8329 | Val   loss 0.3051, acc 0.9101
Epoch 4/30: Train loss 0.4521, acc 0.8489 | Val   loss 0.2990, acc 0.9121
Epoch 5/30: Train loss 0.4311, acc 0.8504 | Val   loss 0.2829, acc 0.9141
Epoch 6/30: Train loss 0.4160, acc 0.8545 | Val   loss 0.2590, acc 0.9303
Epoch 7/30: Train loss 0.4340, acc 0.8459 | Val   loss 0.2470, acc 0.9253
Epoch 8/30: Train loss 0.4345, acc 0.8454 | Val   loss 0.2904, acc 0.8990
Epoch 9/30: Train loss 0.4257, acc 0.8519 | Val   loss 0.2590, acc 0.9131
Epoch 10/30: Train loss 0.4145, acc 0.8517 | Val   loss 0.2287, acc 0.9343
Epoch 11/30: Train loss 0.4035, acc 0.8573 | Val   loss 0.2307, acc 0.9202
Epoch 12/30: Train loss 0.4304, acc 0.8465 | Val   loss 0.2216, acc 0.9182
Epoch 13/30: Train loss 0.3932, acc 0.8578 | Val   loss 0.2636, acc 0.9051
Epoch 14/30: Tr